In [1]:
import numpy as np
from random import random
from functools import reduce
from queue import PriorityQueue, SimpleQueue, LifoQueue
from collections import namedtuple

In [32]:
PROBLEM_SIZE = 50
NUM_SETS = 200
# Arrays where all elements have a probability of 20% of being true (Basic building blocks to cover the set)
SETS = tuple(np.array([random() < .3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple('State', ['taken', 'not_taken'])

In [33]:
def goal_check(state : State) :
    # Combination of every chosen blocks and checking if all is covered --> Test for goal state
    return np.all(reduce(np.logical_or, (SETS[i] for i in state[0]), np.array([False for _ in range(PROBLEM_SIZE)])))

In [34]:
# If the whole set can't be covered even with all the building sets, the problem can't be solved
assert goal_check(State(set(range(NUM_SETS)),set())), "Problem not solvable"

Solving the problem with uninformed strategy

In [5]:
# Change PriorityQueue with SimpleQueue to have Breadth-first, and LifoQueue to have Depth-first
frontier = SimpleQueue()
# Initial state
frontier.put(State(set(), set(range(NUM_SETS))))

current_state = frontier.get()
counter = 0
while not goal_check(current_state) : 
    counter += 1
    for a in current_state[1] :
        new_state = State(current_state[0] ^ {a}, current_state[1] ^ {a})
        frontier.put(new_state)
    current_state = frontier.get()
    
print("Solved in", counter, "steps :", current_state.taken)

Solved in 11 steps : {0, 1}


Solving the problem with A*

In [36]:
def h(state: State, set_size):
    return set_size - np.sum(
        reduce(
            np.logical_or,
            (SETS[i] for i in state.taken),
            np.array([False for _ in range(set_size)]),
        )
    )

# h_alt has the same initial value as h, but it is decreased by the number of uncovered elements that simultaneously can be covered by one tile
def h_alt(state: State, set_size) :
    initial_cost = h(state, set_size)
    missing_elements = [index for index,value in enumerate(reduce(np.logical_or, (SETS[i] for i in state.taken), np.array([False for _ in range(set_size)]))) if not value]
    max_covered_with_one_set = 0
    for set in [SETS[i] for i in state.not_taken] :
        cnt_covered_elems = 0
        for i, value in enumerate(set) :
            if not value :
                continue
            if i in missing_elements :
                cnt_covered_elems += 1
        if cnt_covered_elems > max_covered_with_one_set :
            max_covered_with_one_set = cnt_covered_elems
    return initial_cost - max_covered_with_one_set + 1

In [30]:
SETS

(array([False,  True, False, False,  True]),
 array([ True, False, False, False, False]),
 array([ True, False, False, False, False]),
 array([False, False, False, False, False]),
 array([False,  True,  True, False, False]),
 array([ True, False, False, False,  True]),
 array([ True, False, False,  True,  True]),
 array([False,  True, False,  True,  True]),
 array([False, False, False, False, False]),
 array([False,  True, False,  True, False]))

In [25]:
s = State({8},{0,1,2,3,4,5,6,7,9})
h_alt(s, 5)

2

In [7]:
def g(state : State) :
    return len(state.taken)

In [37]:
frontier = PriorityQueue()

cur_state = State(set(), set([i for i in range(NUM_SETS)]))
# frontier.put((0,cur_state))

counter_steps = 0
while cur_state is not None and not goal_check(cur_state) :
    counter_steps += 1
    for action in cur_state.not_taken :
        new_state = State(cur_state.taken ^ {action}, cur_state.not_taken ^ {action})
        new_cost = g(new_state) + h_alt(new_state, PROBLEM_SIZE)
        frontier.put((new_cost, new_state))
    _, cur_state = frontier.get()

if cur_state is None :
    print("No solution was found :(")
else :
    print("Solution found :", cur_state.taken,"in", counter_steps, "steps")

Solution found : {88, 177, 99, 57} in 204 steps
